In [1]:
df = spark.sql("SELECT * FROM bing_lake_db.latest_new_tbl")
display(df)

StatementMeta(, 5226aa70-ed38-4ba5-8987-cb42312ad0c5, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1f70daf1-2456-44cd-9354-d53278c13160)

In [2]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 5226aa70-ed38-4ba5-8987-cb42312ad0c5, 4, Finished, Available, Finished)

In [3]:
model=(AnalyzeText()
       .setTextCol("snippet")
       .setKind("SentimentAnalysis")
       .setOutputCol("response")
       .setErrorCol("error")
)

StatementMeta(, 5226aa70-ed38-4ba5-8987-cb42312ad0c5, 5, Finished, Available, Finished)

In [4]:
result=model.transform(df)

StatementMeta(, 5226aa70-ed38-4ba5-8987-cb42312ad0c5, 6, Finished, Available, Finished)

In [5]:
display(result)

StatementMeta(, 5226aa70-ed38-4ba5-8987-cb42312ad0c5, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b69abe94-9aec-4a9d-9084-a9fa3e407a1f)

In [6]:
from pyspark.sql.functions import col
sentiment_df=result.withColumn("sentiment",col("response.documents.sentiment"))
display(sentiment_df)

StatementMeta(, 5226aa70-ed38-4ba5-8987-cb42312ad0c5, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8c9d785d-52f3-4785-a6a1-ba3a8f3d4167)

In [7]:
sentiment_final_df=sentiment_df.drop("error","response")
display(sentiment_final_df)

StatementMeta(, 5226aa70-ed38-4ba5-8987-cb42312ad0c5, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c63ff9e6-64b7-42d3-af23-044c1f48d85f)

In [8]:
from pyspark.sql.utils import AnalysisException

try:
    table_name='bing_lake_db.latest_sentiment_analysis'
    sentiment_final_df.write.format("delta").saveAsTable(table_name)
except:
    print("Table Already Exists!")
    sentiment_final_df.createOrReplaceTempView("vw_sentiment_final_df")

    spark.sql(f"""
    MERGE INTO bing_lake_db.latest_sentiment_analysis AS target_table
    USING vw_sentiment_final_df AS source_view
    ON source_view.link = target_table.link
    WHEN MATCHED AND (
    source_view.source <> target_table.source OR
    source_view.title <> target_table.title OR
    source_view.snippet <> target_table.snippet OR
    source_view.link <> target_table.link OR
    source_view.date = target_table.date OR
    source_view.day <> target_table.day OR
    source_view.month <> target_table.month OR
    source_view.year <> target_table.year OR
    source_view.hour <> target_table.hour OR
    source_view.weekday <> target_table.weekday
    )
    THEN UPDATE SET *
    
    WHEN NOT MATCHED THEN INSERT *
    """)

StatementMeta(, 5226aa70-ed38-4ba5-8987-cb42312ad0c5, 10, Finished, Available, Finished)

Table Already Exists!


In [9]:
%%sql

SELECT * from bing_lake_db.latest_sentiment_analysis

StatementMeta(, 5226aa70-ed38-4ba5-8987-cb42312ad0c5, 11, Finished, Available, Finished)

<Spark SQL result set with 195 rows and 11 fields>